In [2]:
from bs4 import BeautifulSoup
import urllib2
import urllib
import json
import csv
import time
import pickle
import pandas as pd

In [151]:
html_pages = load_obj("angel_html_c_full")

In [ ]:
def backup(obj, iter_num):
    name = "fin_data_" + str(iter_num)
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
request_headers = {"Host": "angel.co",
"Connection": "keep-alive",
"Content-Length": "18",
"Origin": "https://angel.co",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36",
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
"Accept": "*/*",
"X-Requested-With": "XMLHttpRequest",
"Referer": "https://angel.co/companies",
"Accept-Language": "en-US,en;q=0.8",
}


start = time.time()
#html_pages = {}
for page in range(3500):
    link = "https://angel.co/company_filters/search_data"
    data = urllib.urlencode({"page":str(page), "sort":"raised"})
    request = urllib2.Request(link, headers=request_headers)
    contents = urllib2.urlopen(request, data=data).read()
    content_d = json.loads(contents)
    beg = "https://angel.co/companies/startups?" + "&".join(["ids%5B%5D=" + str(id) for id in content_d['ids']])
    end = "&total=" + str(content_d["total"]) + "&page=" + str(content_d["page"]) + "&sort=" + str(content_d["sort"]) + "&new=" + str(content_d["new"]) + "&hexdigest=" + str(content_d["hexdigest"])
    link = beg + end
    html_pages[page] = urllib2.urlopen(link).read()
    
    time.sleep(1)
    if page % 100 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * float(page) / 3500) + ' running for ' + str(mins) + ' mins'
        backup(html_pages, page)

In [ ]:
request_headers = {"Host": "angel.co",
"Connection": "keep-alive",
#"Content-Length": "100",
"Origin": "https://angel.co",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36",
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
"Accept": "*/*",
"X-Requested-With": "XMLHttpRequest",
"Referer": "https://angel.co/companies",
"Accept-Language": "en-US,en;q=0.8",
"DNT": "1"}

start = time.time()
html_pages_c = []
vals = range(0, 100000000, 100000)
max_val = vals[1:]
min_val = vals[0:-1]
base_url = "https://angel.co/company_filters/search_data" 
print 'Starting'
for i in range(999):
    min_v, max_v = min_val[i], max_val[i]
    for page in range(20):
        data="filter_data%5Braised%5D%5Bmin%5D=" + str(min_v) + "&filter_data%5Braised%5D%5Bmax%5D=" + str(max_v) + "&sort=signal&page=" + str(page)
        request = urllib2.Request(base_url, headers=request_headers)
        contents = urllib2.urlopen(request, data=data).read()
        content_d = json.loads(contents)
        if content_d['ids'] != []:
            beg = "https://angel.co/companies/startups?" + "&".join(["ids%5B%5D=" + str(id) for id in content_d['ids']])
            end = "&total=" + str(content_d["total"]) + "&page=" + str(content_d["page"]) + "&sort=" + str(content_d["sort"]) + "&new=" + str(content_d["new"]) + "&hexdigest=" + str(content_d["hexdigest"])
            link = beg + end
            html_pages_c.append(urllib2.urlopen(link).read())
        else:
            break
        time.sleep(0.1)
    if i % 50 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * i / 50.0) + ' running for ' + str(mins) + ' mins'
        backup(html_pages_c, max_v)
backup(html_pages_c, "full")

In [152]:
# All possible fields

soup = BeautifulSoup(html_pages[1], "lxml")
soup.find_all("div", class_="pitch")[0]
soup.find_all("div", class_="value")[0]
soup.find_all("div", class_="market")[0]
soup.find_all("div", class_="website")[0]
soup.find_all("div", class_="joined")[0]
soup.find_all("div", class_="location")[0]
soup.find_all("div", class_="company_size")[0]
soup.find_all("div", class_="stage")[0]
soup.find_all("div", class_="raised")[0]

<div class="column hidden_column raised sortable" data-column="raised">\\nTotal Raised\\n</div>

In [155]:
len_pages = len(html_pages)
print 'Starting'
start = time.time()
names, raised, pitch, stage, market = [], [], [], [], []
for i in range(1, len_pages): 
    soup = BeautifulSoup(html_pages[i], "lxml")
    names_unf = soup.find_all("div", class_="name")
    raised_unf = soup.find_all("div", class_="raised")
    pitch_unf = soup.find_all("div", class_="pitch")
    stage_unf = soup.find_all("div", class_="stage")
    market_unf = soup.find_all("div", class_="market")
    if i % 100 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * float(i) / len_pages) + ' running for ' + str(mins) + ' mins'
    
    res_len = min(len(names_unf), len(raised_unf), len(pitch_unf), 
        len(stage_unf), len(market_unf))
    for j in range(res_len): 
        names.append(names_unf[j].text[2:-2])
        raised.append(raised_unf[j].text[22:-4])
        pitch.append(pitch_unf[j].text[2:-2])
        stage.append(stage_unf[j].text[15:-4])
        market.append(market_unf[j].text[16:-4])

startup_data_full = {"Names":names, "Raised": raised, "Pitch":pitch, "Stage":stage, "Market":market} 

Starting
Percent progress 3.34784064279 running for 0.221906415621 mins
Percent progress 6.69568128557 running for 0.435572417577 mins
Percent progress 10.0435219284 running for 0.655195017656 mins
Percent progress 13.3913625711 running for 0.875916063786 mins
Percent progress 16.7392032139 running for 1.09028981527 mins
Percent progress 20.0870438567 running for 1.30546389818 mins
Percent progress 23.4348844995 running for 1.5151494503 mins
Percent progress 26.7827251423 running for 1.7292738835 mins
Percent progress 30.1305657851 running for 1.9336484313 mins
Percent progress 33.4784064279 running for 2.13388544718 mins
Percent progress 36.8262470706 running for 2.34065020084 mins
Percent progress 40.1740877134 running for 2.5415830493 mins
Percent progress 43.5219283562 running for 2.73126256466 mins
Percent progress 46.869768999 running for 2.91019073327 mins
Percent progress 50.2176096418 running for 3.08468063275 mins
Percent progress 53.5654502846 running for 3.25285019875 mins


In [212]:
names = [name.replace(' ', '-') for name in names]
startup_data_full = {"Names":names, "Raised": raised, "Pitch":pitch, "Stage":stage, "Market":market} 
startup_df = pd.DataFrame(startup_data_full)
startup_df.head()

,Market,Names,Pitch,Raised,Stage
0,,Unwind-Me,"Massage Made Simple: The best therapists, stra...",,
1,Marketplaces,Jackson-Square-Ventures,VC firm investing in SaaS and marketplaces.,-,
2,SaaS,Contour-Ventures,,-,-
3,-,GSV-Capital,,-,-
4,Financial Services,Vast-Ventures,Investing to improve human potential,-,-


In [195]:
comp_data = {}
companies = list(startup_df['Names'].unique())
companies = [company.replace(' ', '-') for company in companies]
error_comps = []
companies_len = len(companies)
for i in range(companies_len):
    try:
        company = companies[i]
        link = "https://angel.co/" + company + "?utm_source=companies"
        soup = BeautifulSoup(urllib2.urlopen(link).read(), "lxml")
        fin_details = soup.find_all("div", class_="details")
        comp_data[company] = {"finance":[]}
        for fin_detail in fin_details:
            fin_round = fin_detail.find_all("div", class_="type")[0].text[1:-1]
            fin_date = fin_detail.find_all("div", class_ ="date_display")[0].text
            fin_amount = fin_detail.find_all("div", class_="raised")
            fin_amount = fin_amount[0].text[1:-1]
            comp_data[company]["finance"].append((fin_round, fin_date, fin_amount))
        prod_desc = soup.find_all("div", class_="product_desc")[0]
        comp_data[company]["desc"] = prod_desc.find_all("div", class_="content")[0].text[1: -1]
        if i % 1000 == 0:
            mins = (time.time() - start) / 60
            print 'Percent progress ' + str(100.0 * i / companies_len) + ' running for ' + str(mins) + ' mins'
            backup(comp_data, companies.index(company))
    except:
        error_comps.append(company)

Percent progress 0.0 running for 20.6441246311 mins
Percent progress 69.3200854948 running for 310.646753867 mins
Percent progress 92.4267806597 running for 405.472500134 mins


In [242]:
fin_data = load_obj('fin_data_full')

def fin_isvalid(data):
    stages = [dets[0] for dets in data]
    if any("Series B" in s for s in stages) or any("Series C" in s for s in stages):
        if stages.count('Series A') < 2 and stages.count('Series B') < 2 and stages.count('Series C') < 2 and stages.count('Series D') < 2:
                return True
    return False
name, fin, desc = [], [], []
for k in fin_data.keys():
    if fin_data[k].has_key('finance'):
        if fin_isvalid(fin_data[k]['finance']):
            name.append(k)
            fin.append(fin_data[k]['finance'])
            if fin_data[k].has_key('desc'):
                desc.append(fin_data[k]['desc'])
            else:
                desc.append('')   

no_stage, seed, series_a, series_b, series_c, series_d = [], [], [], [], [], []
for fin_list in fin:
    c_no_stage, c_seed, c_series_a, c_series_b, c_series_c, c_series_d = [], [], [], [], [], []
    for fin_round in fin_list:
        if fin_round[0] == 'Seed':
            c_seed.append(fin_round)
        elif fin_round[0] == 'No Stage':
            c_no_stage.append(fin_round)
        elif fin_round[0] == 'Series A':
            c_series_a.append(fin_round)
        elif fin_round[0] == 'Series B':
            c_series_b.append(fin_round)
        elif fin_round[0] == 'Series C':
            c_series_c.append(fin_round)
        elif fin_round[0] == 'Series D':
            c_series_d.append(fin_round)
    no_stage.append(c_no_stage)
    seed.append(c_seed)
    series_a.append(c_series_a)
    series_b.append(c_series_b)
    series_c.append(c_series_c)
    series_d.append(c_series_d)
for funding in [seed, no_stage, series_a, series_b, series_c, series_d]:
    for i in range(len(funding)):
        ele = funding[i]
        if ele == []:
            funding[i] = ('', '', '')
        else:
            funding[i] = ele[0]
            
seed_amount = [val[2] for val in seed]
no_stage_amount = [val[2] for val in no_stage]
series_a_amount = [val[2] for val in series_a]
series_b_amount = [val[2] for val in series_b]
series_c_amount = [val[2] for val in series_c]
series_d_amount = [val[2] for val in series_d]

seed_date = [val[1] for val in seed]
no_stage_date = [val[1] for val in no_stage]
series_a_date = [val[1] for val in series_a]
series_b_date = [val[1] for val in series_b]
series_c_date = [val[1] for val in series_c]
series_d_date = [val[1] for val in series_d]

market, pitch, stage = [], [], []
for comp in name:
    companydf = startup_df[startup_df["Names"] == comp]
    market.append(list(companydf.Market)[0])
    pitch.append(list(companydf.Pitch)[0])
    stage.append(list(companydf.Stage)[0])

funding_full = {"Names":name, "Market": market, "Pitch": pitch, "Stage": stage, "Description":desc,
                "No_Stage_Date": no_stage_date, "No_Stage_Amount":no_stage_amount,                
                "Seed_Date": seed_date, "Seed_Amount":seed_amount,
                "Series_A_Date": series_a_date, "Series_A_Amount":series_a_amount, 
                "Series_B_Date":series_b_date, "Series_B_Amount":series_b_amount, 
                "Series_C_Date":series_c_date, "Series_C_Amount":series_c_amount,
                "Series_D_Date":series_d_date, "Series_D_Amount":series_d_amount} 
funding_df = pd.DataFrame(funding_full)
funding_df = funding_df[funding_df["Stage"] != "Acquired"]
funding_df.head()

,Description,Market,Names,No_Stage_Amount,No_Stage_Date,Pitch,Seed_Amount,Seed_Date,Series_A_Amount,Series_A_Date,Series_B_Amount,Series_B_Date,Series_C_Amount,Series_C_Date,Series_D_Amount,Series_D_Date,Stage
1,,Cable,Epic-Sciences,,,,,,,,"$13,000,000","Nov 13, 2012","$30,000,000","Jul 30, 2014",,,Series C
2,,All Students,Apreso-Classroom,,,,,,,,"$15,000,000","Oct 14, 2008",,,,,Series B
3,Visualead (视觉码) creates better interactions be...,Bridging Online and Offline,Visualead,,,Effective and Secure Offline to Mobile experie...,"$750,000","Mar 25, 2012","$1,600,000","Aug 15, 2013",Unknown,"Jan 20, 2015",,,,,Series B
4,,Food Processing,Onshift,"$7,000,000","Feb 3, 2014",,,,,,"$3,000,000","Feb 2, 2012",,,,,Series C
5,,-,Xendex-Holding,,,,,,Unknown,"Jun 25, 2008","$3,500,000","Nov 30, 2009",,,,,Series A


In [3]:
funding_df=pd.read_csv("funding.csv")
funding_df.fillna('', inplace=True)
funding_df.head(10)

,Description,Market,Names,No_Stage_Amount,No_Stage_Date,Pitch,Seed_Amount,Seed_Date,Series_A_Amount,Series_A_Date,Series_B_Amount,Series_B_Date,Series_C_Amount,Series_C_Date,Series_D_Amount,Series_D_Date,Stage
0,,Cable,Epic-Sciences,,,,,,,,"$13,000,000","Nov 13, 2012","$30,000,000","Jul 30, 2014",,,Series C
1,,All Students,Apreso-Classroom,,,,,,,,"$15,000,000","Oct 14, 2008",,,,,Series B
2,Visualead (视觉码) creates better interactions be...,Bridging Online and Offline,Visualead,,,Effective and Secure Offline to Mobile experie...,"$750,000","Mar 25, 2012","$1,600,000","Aug 15, 2013",Unknown,"Jan 20, 2015",,,,,Series B
3,,Food Processing,Onshift,"$7,000,000","Feb 3, 2014",,,,,,"$3,000,000","Feb 2, 2012",,,,,Series C
4,,-,Xendex-Holding,,,,,,Unknown,"Jun 25, 2008","$3,500,000","Nov 30, 2009",,,,,Series A
5,Palo Alto-based FilmLoop has reportedly layed ...,Software,filmloop,,,,,,"$5,600,000","Feb 1, 2005","$7,000,000","May 1, 2006",,,,,Series A
6,,,Mochi-Media,,,,,,"$4,000,000","Mar 12, 2008","$10,000,000","Jun 18, 2008",,,,,
7,,-,SkyRecon-Systems,,,,,,"$3,730,000","Sep 12, 2005","$6,500,000","Mar 12, 2007",,,,,Series A
8,Challenge your brain with games designed by ne...,,Lumos-Labs,,,Creator of Lumosity,"$450,000","Jun 11, 2007","$3,100,000","Jun 3, 2008",,,"$32,500,000","Jun 16, 2011","$31,500,000","Aug 22, 2012",
9,"ROBLOX is a comprehensive 3D creation, publish...",Blockchains,ROBLOX,,,User-Generated Online Gaming Platform,,,"$2,200,000","Aug 14, 2009","$4,000,000","Jun 14, 2011",,,,,Series A


In [ ]:
from bs4 import BeautifulSoup
import urllib2
import urllib
import json
import csv
import time
import pickle

request_headers = {
"Connection": "keep-alive",
"Content-Length": "18",
"User-Agent": 
"Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; .NET CLR 2.7.58687; SLCC2; Media Center PC 5.0; Zune 3.4; Tablet PC 3.6; InfoPath.3)",
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
"Accept": "*/*",
"X-Requested-With": "XMLHttpRequest",
"Accept-Language": "en-US,en;q=0.8",
"Cookie":"multivariate_bot=false; D_SID=18.111.93.191:XQebItn+Xbbs4fxri7rWUENSpuNpcQ10POuRhCUaoqA; ahoy_visitor=85c0aa9d-5741-41b0-90a1-66c2407480de; _gat=1; _gat_newTracker=1; __qca=P0-280926842-1448694800387; __uvt=; amplitude_idcrunchbase.com=eyJkZXZpY2VJZCI6ImZlMjZjYmRmLTBjOTMtNGRhYy04ZmFlLTAyMjA1N2M0MzE1ZiIsInVzZXJJZCI6bnVsbH0=; _ga=GA1.2.961691226.1448694800; D_PID=C2974EEC-C295-3B19-881B-3987D348B229; D_IID=A7BBC9FB-58FB-3614-BC42-DC78B9DE46E3; D_UID=176CC3F7-AB55-33C4-934C-F4E77DFD0092; D_HID=3oV+ttwuR6fu7G+eqiUjkdE6kogcj3jL6jqRRPj1Gsw; s_pers=%20s_fid%3D76E928CEE4BFCC75-18BBFE4C3FC0792F%7C1511853228786%3B%20s_getnr%3D1448694828797-New%7C1511766828797%3B%20s_nrgvo%3DNew%7C1511766828800%3B; s_sess=%20s_cc%3Dtrue%3B%20s_sq%3D%3B; uvts=3rLFEsKv57hLKgqX; ahoy_visit=3bfd8d5a-8f74-44ba-b962-9e3ec8223f7e; RT=sl=10&ss=1448694798233&tt=10870&obo=1&bcn=%2F%2F36cc248b.mpstat.us%2F&sh=1448695098621%3D10%3A1%3A10870%2C1448694830134%3D9%3A1%3A10840%2C1448694829092%3D8%3A1%3A10432%2C1448694826377%3D7%3A1%3A8406%2C1448694825821%3D6%3A1%3A8036&dm=crunchbase.com&si=60d1ff9b-5c80-4211-9ed3-061f33fd2b83&r=https%3A%2F%2Fwww.crunchbase.com%2Forganization%2Friskiq&ul=1448695104730"
}


cb_request_headers = {"Host": "www.crunchbase.com",
"Connection": "keep-alive",
"Cache-Control": "max-age=0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36",
"DNT": "1",
"Referer": "https://www.crunchbase.com/organization/riskiq",
"Accept-Encoding": "gzip, deflate, sdch",
"Accept-Language": "en-US,en;q=0.8"}

link = "https://www.crunchbase.com/organization/riskiq"
request = urllib2.Request(link, headers=cb_request_headers)
contents = urllib.urlopen(link).read()

We creates headers the proper headers for accessing the data from Crunchbase and try to make a request. However, after running, this attempt to scrape the data does not work because of antiscraping software. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#cb_data = {}
start = time.time()
org_list = ["riskiq", "pixalate"]
num_orgs = len(org_list)
browser = webdriver.Firefox()
#browser = webdriver.Chrome('/Users/candokevin/Downloads/chromedriver')

for i in range(num_orgs):
    org = org_list[i]
    uri = "https://www.crunchbase.com/organization/" + org
    browser.get(uri)
    wait = WebDriverWait(browser, 30)
    element = wait.until(EC.element_to_be_clickable((By.ID,'anchor-title')))
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "lxml")
    fund_table = soup.find_all("div", class_="funding_rounds")
    fund_table_rows = fund_table[0].find_all("tr")    
    description = soup.find_all("div", class_="description-ellipsis")
    cb_data[org] = (description[0].text, [fund_table_rows[i].text for i in range(1, len(fund_table_rows))])
    
    # Timing and saving code
    if i % 1 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100 * float(i) / num_orgs) + ' running for ' + str(mins) + ' mins'
        backup(cb_data, i)
    time.sleep(5)
        
browser.quit()

Also attempted to run scrapers using the external library Selenium. However, Crunchbase has anti-scraping software that prevents you from using these types of methods. We requested access to the Crunchbase API, but never heard back. Consequently, we used Angel data instead.

In [3]:
%matplotlib inline
from bs4 import BeautifulSoup
import urllib2
import urllib
import json
import csv
import time
import pickle
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import sys
import os
import time
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def merge_d(x, y):
    z = x.copy()
    z.update(y)
    return z

def flatten_data(t_data):
    t_data_f= {}
    for key in t_data.keys():
        for key2 in t_data[key].keys():
            t_data_f[key + '_' + key2] = t_data[key][key2]
    return t_data_f

twit_data = flatten_data(load_obj('twit_data1'))
t_data_num = 1

def backup(obj, t_data_num = t_data_num):
    name = "t_df" + str(t_data_num)
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

We now want to do some data wrangling to aggregate all the data from the Tweets. We want to get information about the tweet text, the number of likes, the number of retweets, names, and href. Later on, we can extract more analytics about these types of interactions.

In [12]:
company_l = []
series_l = []
text_l = [] 
time_l = []
retweets_l = []
likes_l = []
name_l = []
href_l = []
errors = []
max_ind = len(twit_data.keys())
start = time.time()

for i in range(max_ind):    
    
    # Timing code and backup code
    if i % 50 == 0:
        mins = (time.time() - start) / 60
        print 'Percent progress ' + str(100.0 * i / max_ind) + ' running for ' + str(mins) + ' mins'          
        data_backup = {'Company': company_l, 'Series': series_l, 'Text': text_l, 'Time':time_l, 'Retweets':retweets_l, 'Likes':likes_l, 'Name':name_l, 'Href':href_l}
        backup(data_backup)
    
    # Finding the series of the data
    company_series = twit_data.keys()[i]
    ind = company_series[:company_series.rfind("_")].rfind("_")
    company = company_series[:ind]
    series = company_series[ind:]
    backup({'Company': company_l, 'Series': series_l, 'Text': text_l, 'Time':time_l, 'Retweets':retweets_l, 'Likes':likes_l, 'Name':name_l, 'Href':href_l})
    
    
    # Loop through each copmany and series pair and extract relevant information
    for tweet in twit_data[company_series]:
        try:
            # Searching
            soup = BeautifulSoup(tweet, 'lxml')
            actions = soup.find_all('div', class_='ProfileTweet-actionCountList')[0]
            text = soup.find_all('p', class_ = 'TweetTextSize')[0].text
            times = soup.find_all('a', class_ = 'tweet-timestamp')[0].text
            retweets = actions.find_all('span', class_='ProfileTweet-actionCount')[0].text.strip()
            likes = actions.find_all('span', class_='ProfileTweet-actionCount')[1].text.strip()
            name = soup.find_all('strong', class_='fullname')[0].text
            href = soup.find_all('a', class_='account-group')[0]['href']
            
            # Storing
            company_l.append(company)
            series_l.append(series)
            text_l.append(text)
            time_l.append(times)
            retweets_l.append(retweets)
            likes_l.append(likes)
            name_l.append(name)
            href_l.append(href)
            
        # Only listen for keyboard exceptions
        except KeyboardInterrupt:
            try:
                sys.exit(0)
            except SystemExit:
                os._exit(0) 
        except:
            errors.append(tweet)

Percent progress 0.0 running for 5.08666038513e-05 mins
Percent progress 5.0 running for 0.929983933767 mins
Percent progress 10.0 running for 2.19964901606 mins
Percent progress 15.0 running for 3.53876201709 mins
Percent progress 20.0 running for 4.88092761834 mins
Percent progress 25.0 running for 6.11316250165 mins
Percent progress 30.0 running for 7.68281498353 mins
Percent progress 35.0 running for 9.68449503183 mins
Percent progress 40.0 running for 11.6382258654 mins
Percent progress 45.0 running for 13.2810905496 mins
Percent progress 50.0 running for 15.5552463651 mins
Percent progress 55.0 running for 17.8359643817 mins
Percent progress 60.0 running for 20.021214668 mins
Percent progress 65.0 running for 22.598205018 mins
Percent progress 70.0 running for 25.0689711491 mins
Percent progress 75.0 running for 28.2786712845 mins
Percent progress 80.0 running for 31.1675331672 mins
Percent progress 85.0 running for 34.0141220649 mins
Percent progress 90.0 running for 37.26210173

After extracting all tweets from the HTML page, we now want to try to generate features that we can regress on. We save the tweets to a file, and then extract features from all of these tweets.

In [5]:
tweets = pd.read_csv(open("../tweets/tweets.csv", 'rU'), encoding='utf-8', engine='c')
tweets_translated = pd.read_csv(open("../startups/translatedTweets.csv", 'rU'), encoding='utf-8', engine='c')
tweets_translated.head()

/Users/candokevin/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Unnamed: 0,Unnamed: 0.1,Company,Href,Likes,Name,Retweets,Series,Text,Time
0,0,0,Occipital,/yehyehgluglu,0 likes,Maria Melo,0 retweets,_Series_A,@rubaums2 mira in the occipital and frontal to...,14 Jul 2011
1,1,2,Occipital,/ArturoMisifu,0 likes,Arturo Quijano,0 retweets,_Series_A,at the height of the occipital head - hurts if...,14 Jul 2011
2,2,5,Occipital,/victorcab,0 likes,Victor Caballero,0 retweets,_Series_A,Tripod – Timer for 360 Views – iPhone? : http:...,14 Jul 2011
3,3,7,Occipital,/electroniko,0 likes,MIGUEL NEGRETE,0 retweets,_Series_A,"Neurona, glia, axon myelin, parietal, temporal...",14 Jul 2011
4,4,8,Occipital,/nomina_bot,0 likes,体野ミナ,0 retweets,_Series_A,Greater occipital nerve:nervus occipitalis maj...,13 Jul 2011


We only take tweets that relate to companies that have 150 or more tweets. We also want to calculate the average number of days it took to reach 200 tweets. 

In [ ]:
# Calculating relative dates based off of dates like Apr, 5 2015. We can use this later
# for calculating average number of tweets per day.
months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
time_sub = []
for row in tweets.iterrows():
    if type(row[1]['Time']) != unicode:
        time_sub.append(0)
    elif len(row[1]['Time'].split(' ')) == 3:
        if row[1]['Time'].split(' ')[1] in months.keys():        
            time_sub.append(int(row[1]['Time'].split(' ')[2]) * 365 + months[row[1]['Time'].split(' ')[1]] * 30 + int(row[1]['Time'].split(' ')[0]))
        else:
            time_sub.append(0)
    else:
        time_sub.append(0)
        
tweets['Date_Num'] = time_sub
tweets = tweets[tweets['Date_Num'] != 0]

valid_data = []
for val in zip(list((tweets.groupby(['Company', 'Series']).size() > 150).index), list((tweets.groupby(['Company', 'Series']).size() > 150).values)):
    if val[1]:
        valid_data.append(val[0])
        
# We also only filter the tweets based off of whether or not there was 150 tweets in the past
valid_tweets = []
for valid_pair in valid_data:
    company = valid_pair[0]
    series = valid_pair[1]
    match_company = tweets[tweets['Company'] == company]
    valid_tweets.append(match_company[match_company['Series'] == series])


In [69]:
# We are now incorporate the translated tweets that Roger generated
valid_tweets_pd = pd.concat(valid_tweets)
for row in tweets_translated.iterrows():
    company = row[1]['Company']
    href = row[1]['Href']
    time = row[1]['Time']

    match_comp = valid_tweets_pd[valid_tweets_pd['Company'] == company]
    match_href = match_comp[match_comp['Href'] == href]
    match_time = match_href[match_href['Time'] == time]
    
    res = list(match_time.index)
    if len(res) == 1:
        index = res[0]
        valid_tweets_pd.set_value(index, 'Text', row[1].Text)

In [71]:
valid_tweets_pd.to_csv('valid_tweets.csv', encoding='utf-8')

In [68]:
len(tweets_translated)

198201

It seems that the total number of tweets is now 2/3 of what it used to be. It seems reasonable to drop this many tweets.

In [ ]:
valid_tweets_pd = pd.concat(valid_tweets)
valid_tweets_pd['Likes'] = valid_tweets_pd.apply(lambda row: int(row['Likes'][0]), axis=1)
valid_tweets_pd['Retweets'] = valid_tweets_pd.apply(lambda row: int(row['Retweets'][0]), axis=1)

Now we want to extract relevant features like the number of likes, retweets, and other relevant features for each company funding round pairing. 

In [224]:
# Calculate average tweets
days = valid_tweets_pd.groupby(['Company', 'Series'])['Date_Num'].max() - valid_tweets_pd.groupby(['Company', 'Series'])['Date_Num'].min()
tweets = valid_tweets_pd.groupby(['Company', 'Series']).size()
avg_tweets = tweets / days

# Calculate total likes
total_likes= valid_tweets_pd.groupby(['Company', 'Series'])['Likes'].sum()

# Calculate total retweets
total_retweets = valid_tweets_pd.groupby(['Company', 'Series'])['Retweets'].sum()
companies = [val[0] for val in list(tweets.index)]
series = [val[1] for val in list(tweets.index)]

# Calculate all features
tweet_features = pd.DataFrame({'Index': range(1, 1+len(days)), 'Company':companies, 'Series':series, 'Total_Likes': total_likes, 'Total_Retweets': total_retweets, 'Avg_Tweets': avg_tweets})
tweet_features.index = range(1, 1+len(days))

# Open old funding data
funding_df=pd.read_csv("funding_collapsed.csv")
funding_df = funding_df.drop(['Unnamed: 0'], 1)
funding_df.rename(columns={'Names':'Company', 'Series_Type':'Series'}, inplace=True)

# Merge the tweet features with the funding data
regress_features = pd.merge(tweet_features, funding_df, how='left', on=['Company', 'Series'])
regress_features = regress_features.drop('Index', 1)
regress_features = regress_features[([type(amount) == str for amount in list(regress_features.Series_Amount)])]
regress_features['Series_Amount'] = regress_features.apply(lambda row: int(row['Series_Amount'][1:].replace(',','')), axis=1)
regress_features.to_csv('regress_features.csv')

In [4]:
%matplotlib inline
from bs4 import BeautifulSoup
import urllib2
import urllib
import json
import csv
import time
import pickle
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import sys
sns.set_style("whitegrid")
sns.set_context("poster")

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

Code for backing up any data that we calculate. We want to make sure that if there is some reason why the twitter data stops being scraped, we save all data scraped thus far.

In [5]:
def backup(obj, info):
    name = "twit_data" + str(info)
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
funding_df=pd.read_csv("funding.csv")
funding_df.fillna('', inplace=True)
funding_df = funding_df.replace(['Unknown'], [''])
funding_df.head(15)

,Description,Market,Names,No_Stage_Amount,No_Stage_Date,Pitch,Seed_Amount,Seed_Date,Series_A_Amount,Series_A_Date,Series_B_Amount,Series_B_Date,Series_C_Amount,Series_C_Date,Series_D_Amount,Series_D_Date,Stage
0,,Cable,Epic-Sciences,,,,,,,,"$13,000,000","Nov 13, 2012","$30,000,000","Jul 30, 2014",,,Series C
1,,All Students,Apreso-Classroom,,,,,,,,"$15,000,000","Oct 14, 2008",,,,,Series B
2,Visualead (视觉码) creates better interactions be...,Bridging Online and Offline,Visualead,,,Effective and Secure Offline to Mobile experie...,"$750,000","Mar 25, 2012","$1,600,000","Aug 15, 2013",,"Jan 20, 2015",,,,,Series B
3,,Food Processing,Onshift,"$7,000,000","Feb 3, 2014",,,,,,"$3,000,000","Feb 2, 2012",,,,,Series C
4,,-,Xendex-Holding,,,,,,,"Jun 25, 2008","$3,500,000","Nov 30, 2009",,,,,Series A
5,Palo Alto-based FilmLoop has reportedly layed ...,Software,filmloop,,,,,,"$5,600,000","Feb 1, 2005","$7,000,000","May 1, 2006",,,,,Series A
6,,,Mochi-Media,,,,,,"$4,000,000","Mar 12, 2008","$10,000,000","Jun 18, 2008",,,,,
7,,-,SkyRecon-Systems,,,,,,"$3,730,000","Sep 12, 2005","$6,500,000","Mar 12, 2007",,,,,Series A
8,Challenge your brain with games designed by ne...,,Lumos-Labs,,,Creator of Lumosity,"$450,000","Jun 11, 2007","$3,100,000","Jun 3, 2008",,,"$32,500,000","Jun 16, 2011","$31,500,000","Aug 22, 2012",
9,"ROBLOX is a comprehensive 3D creation, publish...",Blockchains,ROBLOX,,,User-Generated Online Gaming Platform,,,"$2,200,000","Aug 14, 2009","$4,000,000","Jun 14, 2011",,,,,Series A


In [7]:
months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
def find_dates(date_str):
    if len(date_str) < 8:
        return None
    yr = date_str[-4:]
    mon_str = date_str[:3]
    mon_num = None
    for k in months.keys():
        if k == mon_str:
            mon_num = months[k]
    if not yr.isdigit() or mon_num is None:
        return None
    mon_num = (mon_num - 1) 
    if mon_num == 0:
        mon_num = 12
    end_date = str(yr) + '-' + str(mon_num) + '-15'
    start_date = str(int(yr) - 10) + '-' + str(mon_num) + '-15'
    return (start_date, end_date)

In [8]:
series_A = funding_df[funding_df['Series_A_Amount'] != ''][['Names', 'Series_A_Amount', 'Series_A_Date']]
series_A.columns = ['Names', 'Series_Amount', 'Series_Date']
series_A['Series_Type'] = pd.Series('Series_A', index=series_A.index)

series_B = funding_df[funding_df['Series_B_Amount'] != ''][['Names', 'Series_B_Amount', 'Series_B_Date']]
series_B.columns = ['Names', 'Series_Amount', 'Series_Date']
series_B['Series_Type'] = pd.Series('Series_B', index=series_B.index)

series_C = funding_df[funding_df['Series_C_Amount'] != ''][['Names', 'Series_C_Amount', 'Series_C_Date']]
series_C.columns = ['Names', 'Series_Amount', 'Series_Date']
series_C['Series_Type'] = pd.Series('Series_C', index=series_C.index)

series_D = funding_df[funding_df['Series_D_Amount'] != ''][['Names', 'Series_D_Amount', 'Series_D_Date']]
series_D.columns = ['Names', 'Series_Amount', 'Series_Date']
series_D['Series_Type'] = pd.Series('Series_D', index=series_D.index)

funding_collapsed = pd.concat([series_A, series_B, series_C, series_D])
funding_collapsed.head()

,Names,Series_Amount,Series_Date,Series_Type
2,Visualead,"$1,600,000","Aug 15, 2013",Series_A
5,filmloop,"$5,600,000","Feb 1, 2005",Series_A
6,Mochi-Media,"$4,000,000","Mar 12, 2008",Series_A
7,SkyRecon-Systems,"$3,730,000","Sep 12, 2005",Series_A
8,Lumos-Labs,"$3,100,000","Jun 3, 2008",Series_A


This is the list of startup funding rounds and their respective dates. We now want to do a query from Twitter for the last 200 tweets before this date of the funding round. The code below does this using a Selenium web drive. We tried using the Twitter API, but it was unsuccesful because Twitter only allows you to scrape the last couple days.

In [69]:
comps = [name.replace('-', '') for name in list(funding_collapsed['Names'])]
series = list(funding_collapsed['Series_Type'])
dates = list(funding_collapsed['Series_Date'])


browser = webdriver.Firefox()
max_iters = len(comps)
start = time.time()

# This was parallelized across multiple notebooks run by multiple people
for i in range(158, 1000):
    try: 
        if i % 50 == 0:
            mins = (time.time() - start) / 60
            print 'Percent progress ' + str(100.0 * i / max_iters) + ' running for ' + str(mins) + ' mins'            
            backup(comp_tweets, i)
        
        # Get 200 tweets per company
        comp_name = comps[i]
        round_fund = series[i]
        date = dates[i]

        start_end_dates = find_dates(date)
        if start_end_dates is None:
            errors.append((comp_name, round_fund, date))
            continue
        else:
            start_date, end_date = start_end_dates

        if not comp_tweets.has_key(comp_name):
            comp_tweets[comp_name] = {}
        if comp_tweets[comp_name].has_key(round_fund):
            errors.append((comp_name, round_fund, date))
            continue

        comp_tweets[comp_name][round_fund] = []

        url = "https://twitter.com/search?q=" + comp_name + "%20since%3A" + start_date + "%20until%3A" + end_date + "&src=typd&lang=en"

        browser.get(url)
        y_pos, y_pos_old = 0, -1
        for j in range(11):           
            if y_pos != y_pos_old:
                wait = WebDriverWait(browser, 30)
                time.sleep(1)
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                y_pos_old = y_pos
                y_pos = int(browser.execute_script("return window.scrollY;"))
        
        html_source = browser.page_source
        soup = BeautifulSoup(html_source, "lxml")

        for tweet in soup.find_all("li"):
            if tweet.get('id') is not None:
                if tweet.get('id')[:17] == 'stream-item-tweet':
                    comp_tweets[comp_name][round_fund].append(str(tweet))
            
    except KeyboardInterrupt:
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0) 
    except:
        errors.append((comp_name, series, date))

Percent progress 3.33667000334 running for 2.48515653213 mins
Percent progress 4.17083750417 running for 13.5679881175 mins
Percent progress 5.00500500501 running for 29.8278889497 mins
Percent progress 5.83917250584 running for 47.3346038342 mins
Percent progress 6.67334000667 running for 60.4890218178 mins
Percent progress 7.50750750751 running for 90.4858969013 mins
Percent progress 8.34167500834 running for 114.817298734 mins
Percent progress 9.17584250918 running for 144.381359633 mins
Percent progress 10.01001001 running for 158.611524598 mins
Percent progress 10.8441775108 running for 177.963723048 mins
Percent progress 11.6783450117 running for 201.168078498 mins
Percent progress 12.5125125125 running for 210.130816066 mins
Percent progress 13.3466800133 running for 231.395539149 mins
Percent progress 14.1808475142 running for 249.466241948 mins
Percent progress 15.015015015 running for 274.641155465 mins
Percent progress 15.8491825158 running for 296.378485099 mins


In [70]:
# Final backup of the data
backup(comp_tweets, 1)